# Spark DataFrames Basics

#### In Apache Spark, a DataFrame is a distributed collection of rows under named columns. In simple terms, it is same as a table in relational database or an Excel sheet with Column headers. It also shares some common characteristics with RDD:

1. Immutable in nature : We can create DataFrame / RDD once but can’t change it. And we can transform a DataFrame / RDD  after applying transformations.
2. Lazy Evaluations: Which means that a task is not executed until an action is performed.
3. Distributed: RDD and DataFrame both are distributed in nature.


#### Advantages

1. DataFrames are designed for processing large collection of structured or semi-structured data.
2. Observations in Spark DataFrame are organised under named columns, which helps Apache Spark to understand the schema of a DataFrame. This helps Spark optimize execution plan on these queries.
3. DataFrame in Apache Spark has the ability to handle petabytes of data.
4. DataFrame has a support for wide range of data format and sources.
5. It has API support for different languages like Python, R, Scala, Java.


In [14]:
from pyspark import SparkContext as sc

In [15]:
from pyspark.sql import SparkSession

In [16]:
spark1 = SparkSession.builder.appName('Basics').getOrCreate()